<a href="https://colab.research.google.com/github/hrbolek/func2pipe/blob/master/notebooks/func2pipe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
def createPipe(*F):
  def Fdiamond(sequence):
    result = sequence
    for Fx in F:
      result = Fx(result)
    return result
  return Fdiamond

In [0]:
def createSub(assign = lambda source, result: {**source, **result}, reducer = None):

    def resultAll(*F):
        buffer = {'data': None}
        def pre(gen):
            for item in gen:
                buffer['data'] = item
                yield item

        def inner(gen):
            e = gen
            for f in F:
                e = f(e)
            return e

        def post(gen):
            for item in gen:
                yield assign(buffer['data'], item)

        return lambda gen: post(inner(pre(gen)))

    def resultOne(*F):
        def inner(gen):
            e = gen
            for f in F:
                e = f(e)
            return e

        def result(gen):
            for item in gen:
                itemResult = reducer(inner([item]))
                yield assign(item, itemResult)
        return result

    if reducer == None:
        return resultAll
    else:
        return resultOne

In [0]:
from functools import wraps
def convertToPipeFunc(with_yield = False, with_state = False):
    def pipeitsimple(func):
        @wraps(func)
        def bindparams(**kwargs):
            def innerselect(generator):
                for i in generator:
                    yield func(i, **kwargs)
            return innerselect
        return bindparams

    if (not with_yield) & (not with_state):
        return pipeitsimple
    
    def hasyield(func):
        @wraps(func)
        def bindparams(**kwargs):
            def pipeFunction(generator):
                for i in generator:
                    for j in func(i, **kwargs):
                        yield j
            return pipeFunction
        return bindparams

    if (with_yield) & (not with_state):
        return hasyield

    def pipeitwithstate(func):
        @wraps(func)
        def bindparams(**kwargs):
            def innerselect(generator):
                state = None
                initState = True
                for i in generator:
                    result = None
                    if (initState):
                        result, state = func(i, **kwargs)
                        initState = False
                    else:
                        result, state = func(i, state = state, **kwargs)
                    yield result

            return innerselect
        return bindparams
    if (not with_yield) & with_state:
        return pipeitwithstate

    def pipeitwithstatewithyield(func):
        @wraps(func)
        def bindparams(**kwargs):
            def innerselect(generator):
                state = None
                initState = True
                for i in generator:
                    result = None
                    if (initState):
                        for result, statel in func(i, **kwargs):
                            state = statel
                            yield result
                        initState = False
                    else:
                        for result, statel in func(i, state = state, **kwargs):
                            state = statel
                            yield result

            return innerselect
        return bindparams
    return pipeitwithstatewithyield

In [8]:
source = [{'value': 0}, {'value': 1}, {'value': 2}]
@convertToPipeFunc()
def add(item, amount):
  return {**item, 'result': item['value'] + amount}

@convertToPipeFunc()
def calc(item, func, name):
  result = {}
  result[name] = func(item)
  return result

@convertToPipeFunc(with_yield = True)
def extra(item, extradata):
  for _ in extradata:
    yield _

def createReducer(accumulator = [], f = lambda item, accumulator: [*accumulator, item]):
  def reducer(gen):
    result = accumulator
    for item in gen:
      result = f(item, result)
    return result
  return reducer

pipe_01 = createPipe(
    add(amount = 2),
    createSub(assign = lambda source, result: {**source, 'value2': result}, reducer = createReducer())(
#    SUB(assign = lambda source, result: {**source, **result})(
#    SUB(assign = lambda source, result: {**source, 'value2': result})(
      calc(func = lambda item: 4, name = 'extraValue'),
      extra(extradata = ['A', 'B', 'C'])
    )
    )
result = list(pipe_01(source))
print(result)


[{'value': 0, 'result': 2, 'value2': ['A', 'B', 'C']}, {'value': 1, 'result': 3, 'value2': ['A', 'B', 'C']}, {'value': 2, 'result': 4, 'value2': ['A', 'B', 'C']}]


In [4]:
greatSource = ['A', 'B', 'C']
smallSource = ['a', 'b', 'c']
fromGreat2Small = {'A': 'a', 'B': 'b', 'C': 'c'}
fromSmall2Great = {'a': 'A', 'b': 'B', 'c': 'C'}

@convertToPipeFunc()
def findSmall(greatItem):
  return fromGreat2Small[greatItem]

@convertToPipeFunc()
def findGreat(smallItem):
  return fromSmall2Great[smallItem]

@convertToPipeFunc()
def identityPipe(item):
  return item

gr = {
    'nodes': {
        'Great': identityPipe(), 
        'Small': identityPipe(),
        },
    'edges': [
        {'from': 'Great', 'to': 'Small', 'relation': findSmall(), 'itemname' : 'little'},
        {'from': 'Small', 'to': 'Great', 'relation': findGreat(), 'itemname' : 'big'},
    ]
    }

def createArrayReducer(accumulator = [], f = lambda item, accumulator: [*accumulator, item]):
  def reducer(gen):
    result = accumulator
    for item in gen:
      result = f(item, result)
    return result
  return reducer


def builder(graph, currentnode, filterq = lambda item: True):
    descriptorpipe = graph['nodes'][currentnode]

    def x(relation):
      innerPipe = builder(graph, relation['to'])
      itemname = relation['to']
      if ('itemname' in relation):
          itemname = relation['itemname']
      return createSub(assign = lambda item, result: {**item, itemname: result}, reducer = createArrayReducer())(innerPipe)
'''
    def x(relation):
        itemname = relation['to']
        if ('itemname' in relation):
            itemname = relation['itemname']
        relationpipe = relation['relation']
        filterq = lambda item: True;
        if ('filter' in relation):
            filterq = relation['filter']
        sub = builder(graph, relation['to'], filterq)

        result = lambda item: {**item, itemname: finisher(sub(relationpipe([item])))}
        return result

    def createagregatorfunc(agregatorarray, itemname):
        def result(data):
            for agregator in agregatorarray:
                data[agregator['itemname']] = agregator['function'](data[itemname])
            return data

        return result
'''
    result = []
    
              
#        pipesub(descriptorpipe),
#        pipeselect(lambda item: {**item[0], **item[1]})
#        ]
    for relation in filter(lambda item: item['from'] == currentnode, graph['edges']):
        result += [x(relation)]
'''        
        if ('reducers' in relation):
            #chybi vyber dat!
            itemname = relation['to']
            if ('itemname' in relation):
                itemname = relation['itemname']
            result += [pipeselect(createagregatorfunc(relation['reducers'], itemname))]
'''
    #result += [pipefilter(filterq)]
    return createPipe(*result)

graphPipe = builder(gr, 'Small')
result = list(graphPipe(smallSource))
print(result)

IndentationError: ignored

In [93]:

greatSource = [{'id': 'A'}, {'id': 'B'}, {'id': 'C'}]
smallSource = [{'id': 'a'}, {'id': 'b'}, {'id': 'c'}]
fromGreat2Small = {'A': 'a', 'B': 'b', 'C': 'c'}
fromSmall2Great = {'a': 'A', 'b': 'B', 'c': 'C'}

@convertToPipeFunc()
def greatToSmallRelation(greatItem):
  print('G:', greatItem)
  return {'idS': fromGreat2Small[greatItem['id']]}

@convertToPipeFunc()
def smallToGreatRelation(smallItem):
  #print('S:', smallItem)
  return {'id': fromSmall2Great[smallItem['id']]}

@convertToPipeFunc(with_yield=True)
def smallToGreatRelation2(smallItem):
  #print('S:', smallItem)
  for i in greatSource:
    yield i
  #return {'idG': fromSmall2Great[smallItem['id']]}

@convertToPipeFunc()
def identityPipe(item, description):
  return {**item, 'description': description}

@convertToPipeFunc(with_yield=True)
def anyToNumbersRelation(item):
  for i in range(1,3):
    #yield {'value': i}
    yield i

@convertToPipeFunc()
def numberRecord(item):
  return {'value': item}

@convertToPipeFunc()
def printItem(item, description):
  print(description, item)
  return item

gr = {
    'nodes': {
        'Great': identityPipe(description = 'Great item'), 
        'Small': identityPipe(description = 'Small item'),
        'Numbers': createPipe(printItem(description = 'preN'), numberRecord(), printItem(description = 'posN')),
        },
    'edges': [
        {'from': 'Great', 'to': 'Small', 'relation': greatToSmallRelation(), 'itemname' : 'little'},
        {'from': 'Small', 'to': 'Great', 'relation': smallToGreatRelation2(), 'itemname' : 'bigBC', 'filter': lambda item: item[id] > 'A'},
        {'from': 'Small', 'to': 'Great', 'relation': smallToGreatRelation2(), 'itemname' : 'bigA', 'filter': lambda item: item[id] <= 'A'},
        {'from': 'Small', 'to': 'Numbers', 'relation': anyToNumbersRelation(), 'itemname' : 'numbers'},
        {'from': 'Great', 'to': 'Numbers', 'relation': anyToNumbersRelation(), 'itemname' : 'numbers',
         'reducers' :[
            {'name': 'count', 
             'selector': lambda item: item['value'], 
             'reducer': lambda data: reduce(lambda x, y: (x + 1), data, 0)}]},
    ]
    }

def createArrayReducer(accumulator = [], f = lambda item, accumulator: [*accumulator, item]):
  def reducer(gen):
    result = accumulator
    for item in gen:
      result = f(item, result)
    return result
  return reducer

def builder(graph, currentnode):
  availableNodes = []
  for node in graph['nodes']:
    availableNodes.append(node)
  availableNodes.remove(currentnode)

  def innerBuilder(graph, currentnode, filterq = lambda item: True):
      descriptorPipe = graph['nodes'][currentnode]

      def x(relation):
        itemname = relation['to']
        if ('itemname' in relation):
            itemname = relation['itemname']
        filterq = lambda item: True
        if ('filter' in relation):
            filterq = relation['filter']

        innerPipe = innerBuilder(graph, relation['to'], filterq)

        sub = createSub(assign = lambda item, result: {**item, itemname: result}, reducer = createArrayReducer())
        return sub(relation['relation'], innerPipe)

      result = [descriptorPipe]
      nodesToRemove = []
      for relation in filter(lambda item: item['from'] == currentnode, graph['edges']):
          nodeName = relation['to']
          nodesToRemove.append(nodeName)
          if nodeName in availableNodes:
            #result += [printItem(description = 'pre'), (x(relation)), printItem(description = 'pos')]
            result += [x(relation)]
          
      for node in nodesToRemove:
        if node in availableNodes:
          availableNodes.remove(node)

      return createPipe(*result)
  return innerBuilder(graph, currentnode)

graphPipe = builder(gr, 'Small')
result = list(graphPipe(smallSource))
print(result)

preN 1
posN {'value': 1, 'something': 'true'}
preN 2
posN {'value': 2, 'something': 'true'}
preN 1
posN {'value': 1, 'something': 'true'}
preN 2
posN {'value': 2, 'something': 'true'}
preN 1
posN {'value': 1, 'something': 'true'}
preN 2
posN {'value': 2, 'something': 'true'}
preN 1
posN {'value': 1, 'something': 'true'}
preN 2
posN {'value': 2, 'something': 'true'}
preN 1
posN {'value': 1, 'something': 'true'}
preN 2
posN {'value': 2, 'something': 'true'}
preN 1
posN {'value': 1, 'something': 'true'}
preN 2
posN {'value': 2, 'something': 'true'}
preN 1
posN {'value': 1, 'something': 'true'}
preN 2
posN {'value': 2, 'something': 'true'}
preN 1
posN {'value': 1, 'something': 'true'}
preN 2
posN {'value': 2, 'something': 'true'}
preN 1
posN {'value': 1, 'something': 'true'}
preN 2
posN {'value': 2, 'something': 'true'}
[{'id': 'a', 'description': 'Small item', 'bigBC': [{'id': 'A', 'description': 'Great item', 'numbers': [{'value': 1, 'something': 'true'}, {'value': 2, 'something': 'true'

In [69]:
a = ['Great', 'Small']
a.remove('Great')
print(a)

['Small']
